# How to get the HiPS from a keyword and load it in Aladin

In this Notebook, we will get HiPS of a given target by using MOC Server

## Import required modules

In [1]:
# Fetch data from a url
from urllib.request import urlopen

# Get a JSON object
import json

# Regex
import re

# Aladin
from ipyaladin import Aladin

## Get the data

In [2]:
target = "Mars"

Get a list of HiPS of bodies (and not sky maps)

In [3]:
url = "http://alasky.cds.unistra.fr/MocServer/query?expr=hips_frame!=equatorial,galactic,ecliptic&hips_frame=*&hips_service_url=*&get=record&fmt=json"

In [4]:
response = urlopen(url)
data = json.loads(response.read().decode('latin-1'))

## Filter the results

We only keep elements that match with the target.
And we want those in the visible spectrum if we can.
We also don't want panoramic views from a rover, or other unrelevant HiPS (elevation, ...).
To allow those, or add more restrictions, you just need to remove or add corresponding terms in the lists of searched words in the regex search. For instance, you can remove elevation from the list, to get HiPS representing the elevation.

If no results are found, we try to get those that are corresponding to the target but aren't meeting the other requirements (not in grayscale, ..).

In [5]:
results = []
lowAccuracyResults = []

for item in data :
    if (not "hips_frame" in item) or re.search("(?i)("+target+")", item["hips_frame"]) is None :
        continue
    lowAccuracyResults.append(item["hips_service_url"])
    if "obs_title" in item and re.search("(?i)(?:dust|thermal|elevation|albedo|rough|topo|grayscale|microwave|rover|panorama|(mineral map))", item["obs_title"]) is not None:
        continue
    if not "obs_description" in item or re.search("(?i)(?:dust|thermal|elevation|albedo|rough|topo|grayscale|microwave|rover|panorama|(mineral map))", item["obs_description"]) is None:       
        results.append(item["hips_service_url"])
if results == [] :
    results = lowAccuracyResults

In [6]:
for result in results :
    print(result)

https://alasky.cds.unistra.fr/Planets/Mars_Viking_MDIM21


## Load it in Aladin

In [7]:
aladin = Aladin(
    coo_frame='body',
    survey=results[0]
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…